In [1]:
import pandas as pd

In [49]:
#preprocessing

lnames = ['movie_id','imdb_id','tmdb_id']
links = pd.read_csv('ml-latest-small/links.csv', sep=',', header=None, names=lnames)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep=',', header=None, names=rnames)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-latest-small/movies.csv', sep=',', header=None, names=mnames)

tnames=['user_id','movie_id','tag','timestamp']
tags = pd.read_csv('ml-latest-small/tags.csv', sep=',', header=None, names=tnames)


In [33]:
links[:5]

,movie_id,imdb_id,tmdb_id
0,movieId,imdbId,tmdbId
1,1,0114709,862
2,2,0113497,8844
3,3,0113228,15602
4,4,0114885,31357


In [98]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,userId,movieId,rating,timestamp
1,1,1,4.0,964982703
2,1,3,4.0,964981247
3,1,6,4.0,964982224
4,1,47,5.0,964983815


In [35]:
movies[:5]

,movie_id,title,genres
0,movieId,title,genres
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [97]:
tags[:5]

,user_id,movie_id,tag,timestamp
0,userId,movieId,tag,timestamp
1,2,60756,funny,1445714994
2,2,60756,Highly quotable,1445714996
3,2,60756,will ferrell,1445714992
4,2,89774,Boxing story,1445715207


In [94]:
#Merging the data, users either rated a movie or tagged it
dataTag = pd.merge(pd.merge(movies, tags), links)
dataRating = pd.merge(pd.merge(movies, ratings), links)

In [116]:
dataTag = dataTag.drop([0])#first row is labels
dataTag[:5]

,movie_id,title,genres,user_id,tag,timestamp,imdb_id,tmdb_id
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,0114709,862
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825,0114709,862
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013,0114709,862
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929,0113497,8844
5,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932,0113497,8844


In [117]:
dataRating = dataRating.drop([0])#first row is labels
dataRating[:5]

,movie_id,title,genres,user_id,rating,timestamp,imdb_id,tmdb_id
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,0114709,862
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,0114709,862
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,0114709,862
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,0114709,862
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,0114709,862


In [121]:
#Each movie average rating
dataRating['rating'] = pd.to_numeric(dataRating['rating'], downcast="float")
dataRating['timestamp'] = dataRating['timestamp'].astype(int)
mean_ratings = dataRating.pivot_table(values='rating', index='title', aggfunc='mean')


In [122]:
mean_ratings[:5]

,rating
title,
'71 (2014),4.0
'Hellboy': The Seeds of Creation (2004),4.0
'Round Midnight (1986),3.5
'Salem's Lot (2004),5.0
'Til There Was You (1997),4.0


In [128]:
#Highest rated movies
top_movies = mean_ratings.sort_values(['rating'], ascending=False)
top_movies[:10]

,rating
title,
Gena the Crocodile (1969),5.0
True Stories (1986),5.0
Cosmic Scrat-tastrophe (2015),5.0
Love and Pigeons (1985),5.0
Red Sorghum (Hong gao liang) (1987),5.0
"Thin Line Between Love and Hate, A (1996)",5.0
Lesson Faust (1994),5.0
Eva (2011),5.0
Who Killed Chea Vichea? (2010),5.0


In [145]:
#removing outliers 
ratings_by_title = dataRating.groupby('title').size()
print(ratings_by_title)

32
title
'71 (2014)                                    1
'Hellboy': The Seeds of Creation (2004)       1
'Round Midnight (1986)                        2
'Salem's Lot (2004)                           1
'Til There Was You (1997)                     2
                                             ..
eXistenZ (1999)                              22
xXx (2002)                                   24
xXx: State of the Union (2005)                5
¡Three Amigos! (1986)                        26
À nous la liberté (Freedom for Us) (1931)     1
Length: 9719, dtype: int64


In [146]:
#titles with atleast 10% of the number of ratings as the most rated movie
min_rating = max(ratings_by_title)*0.1
min_rating = int(min_rating)
print(min_rating)
active_titles = ratings_by_title.index[ratings_by_title >= 10]
active_titles[:5]

32


Index([''burbs, The (1989)', '(500) Days of Summer (2009)',
       '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)',
       '10,000 BC (2008)'],
      dtype='object', name='title')

In [147]:
mean_ratings = mean_ratings.loc[active_titles]

mean_ratings[:5]

,rating
title,
"'burbs, The (1989)",3.176471
(500) Days of Summer (2009),3.666667
10 Cloverfield Lane (2016),3.678571
10 Things I Hate About You (1999),3.527778
"10,000 BC (2008)",2.705882
